In [1]:
import requests
import os
import logging
import gdown
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
from torchvision.datasets import VOCSegmentation
import torchmetrics
import torchvision
import albumentations as A

import re
import string
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import cv2
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from scipy.io import loadmat
from sklearn.manifold import TSNE
from torchmetrics.classification import MulticlassF1Score, JaccardIndex, MulticlassPrecision, MulticlassRecall, MulticlassAveragePrecision
import pandas as pd
from torchinfo import torchinfo

from transformers import ConvNextV2Model, BertModel, BertTokenizer, ViTModel, ViTConfig
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, CLIPModel, CLIPTokenizer, CLIPProcessor
from transformers import DeiTConfig, DeiTFeatureExtractor, DeiTImageProcessor, DeiTModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import timm


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
logging.basicConfig(level=logging.ERROR)

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


In [3]:
ROOT_DIR = '../Datasets/ocular-disease-recognition-odir5k/'

## ALIGN

In [4]:
BATCH_SIZE = 16

In [5]:
CSV_PATH = ROOT_DIR + 'dataset_single_eye.csv'
TEST_CSV = ROOT_DIR + 'TESTING_dataset_single_eye.csv'
IMG_PATH = ROOT_DIR + 'preprocessed_images/'

In [6]:
torchvision.io.read_image(IMG_PATH + '0_left.jpg').shape

torch.Size([3, 512, 512])

In [7]:
train_val_df = pd.read_csv(CSV_PATH)
test_df = pd.read_csv(TEST_CSV)

In [8]:
def preprocess_text(df:pd.DataFrame):
    df['Keywords'] = df['Keywords'].str.lower()
    df['Keywords'] = df['Keywords'].apply(lambda x: " ".join(x.split()))
    df['Keywords'] = df['Keywords'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
    return df
train_val_df = preprocess_text(train_val_df)
test_df = preprocess_text(test_df)

In [9]:
np.max(train_val_df['Keywords'].apply(lambda x: len(x.split())))

10

In [10]:
train_df, val_df = train_test_split(train_val_df, test_size = 0.15)
len(train_df), len(val_df)

(4877, 861)

In [69]:
IMG_SIZE = (299, 299)

rescale_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(IMG_SIZE, antialias = True, interpolation = 3),
    torchvision.transforms.Normalize(
        timm.data.constants.IMAGENET_DEFAULT_MEAN,
        timm.data.constants.IMAGENET_DEFAULT_STD
    )
])

In [70]:
train_val_df['Keywords'][5]

'moderate non proliferative retinopathy'

In [71]:
processor = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'roberta-base')

Using cache found in C:\Users\krish/.cache\torch\hub\huggingface_pytorch-transformers_main


In [72]:
class ODIRDatasetMM(Dataset) :
    def __init__(self, df, IMG_FOLDER, tokenizer = processor, feature_extractor = rescale_transform) :
        '''
        id : list of samples ids as string
        '''
        self.text = [tokenizer(text = x, padding = 'max_length', max_length = 45, truncation = True, return_tensors = 'pt') for x in df['Keywords']]
        self.eye = df['eye']
        self.labels = torch.tensor(df[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].to_numpy()).float()
        self.img_dir = [IMG_PATH + x for x in df['Image']]

        self.images = [feature_extractor(torchvision.io.read_image(x).float()) for x in self.img_dir]
        #self.transform = transform
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        return self.images[idx], self.text[idx], self.labels[idx]

In [73]:
train_dataset = ODIRDatasetMM(train_df, IMG_PATH)
val_dataset   = ODIRDatasetMM(val_df, IMG_PATH)
test_dataset  = ODIRDatasetMM(test_df, IMG_PATH)

In [74]:
train_dataloader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size= BATCH_SIZE)

In [75]:
# m = torchvision.models.efficientnet_b0(weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT).to(device)
# torchinfo.summary(m)

In [77]:
# contrastive learning on training data finetuning

In [78]:
# prepare two models: BERT vs ConvNext, try to compute contrastive losses
class ContrastiveLearning(nn.Module):
    def __init__(self, drop_prob = 0.4):
        super().__init__()
        self.img_model = torchvision.models.efficientnet_b0(weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT) #torch.hub.load('facebookresearch/deit:main', 'deit_large_patch16_224', pretrained=True) #output 768 features
        self.img_model.classifier = nn.Sequential(
            nn.Linear(1280, 768)
        )                                                   
        
        self.txt_model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'roberta-base')         #output 768 features
        
        # image model classification head
        self.fc1 = nn.Linear(768, 768)
        self.fc2 = nn.Linear(768, 768)
        self.img_head = nn.Linear(768, 8)
        
        # text model classification ehad
        self.fc3 = nn.Linear(768, 768)
        self.fc4 = nn.Linear(768, 768)
        self.text_head = nn.Linear(768, 8)

        self.dropout_layer = nn.Dropout(drop_prob)

    def forward(self, img_input = None, input_ids = None, attn_mask = None, contrastive = False):#, text_class = False):
        if(contrastive):
            # pretraining
            out_txt = self.txt_model(input_ids, attn_mask)['pooler_output']
            out_img = self.img_model(img_input)

            out_txt_ret = F.normalize(out_txt, p = 2.0, dim = 1)
            out_img_ret = F.normalize(out_txt, p = 2.0, dim = 1)

            #img path
            resi_img = out_img
            img_route_out = self.dropout_layer(out_img)
            img_route_out = F.relu(self.fc1(img_route_out))
            img_route_out = self.dropout_layer(img_route_out)
            img_route_out = F.relu(self.fc2(img_route_out))
            img_route_out = img_route_out + resi_img

            img_route_out = F.sigmoid(self.img_head(out_img))

            #text path
            resi_txt = out_txt
            txt_route_out = self.dropout_layer(out_txt)
            txt_route_out = F.relu(self.fc3(txt_route_out))
            txt_route_out = self.dropout_layer(txt_route_out)
            txt_route_out = F.relu(self.fc4(txt_route_out))
            txt_route_out = txt_route_out + resi_txt

            txt_route_out = F.sigmoid(self.text_head(out_txt))
            return out_img_ret, out_txt_ret, img_route_out, txt_route_out

            #return out_img, out_txt
        else:
            #if(not text_class):
            out = self.img_model(img_input)
            resi = out
            out = self.dropout_layer(out)
            out = F.relu(self.fc1(out))
            out = out + resi

            out = F.sigmoid(self.img_head(out))
            return out
            # else:
            #     out = self.txt_model(input_ids, attn_mask)['pooler_output']
            #     resi = out
            #     out = self.dropout_layer(out)
            #     out = F.relu(self.fc2(out))
            #     out = resi + out

            #     out = F.sigmoid(self.text_head(out))
            #     return out

In [90]:
def contrastive_loss(ten1, ten2, temperature = nn.Parameter(torch.tensor(.25).to(device))):    #...
    #steps = hadamard product
    # trivial for loop 
    sim = torch.einsum('i d, j d -> i j', ten1, ten2) * temperature.exp()
    labels = torch.arange(ten1.size(0), device = device)
    loss = (F.cross_entropy(sim, labels) + F.cross_entropy(sim.t(), labels)) / 2
    return loss.mean()

In [91]:
model = ContrastiveLearning().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
weights = torch.tensor([1,1.2,1.5,1.5,1.5,1.5, 1.5, 1.2]).to(device)
criterion_text = nn.BCELoss(weights)
criterion_image = nn.BCELoss(weights)
cont_loss = contrastive_loss
AVERAGING = 'micro'
acc_train_img = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
acc_train_text = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
acc_val_img   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)
acc_val_text   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)

EPOCHS = 25

for epoch_num in range(EPOCHS):

      total_acc_train = 0
      total_loss_train = 0

      for train_image, train_text, train_label in tqdm(train_dataloader):
          optimizer.zero_grad()

          train_label = train_label.to(device)
          train_image = train_image.to(device)
          mask = train_text['attention_mask'].to(device)
          input_id = train_text['input_ids'].squeeze(1).to(device)
          
          # logits_per_image, logits_per_text
          out_img, out_txt, predictions_img, predictions_text = model.forward(train_image, input_id, mask, contrastive = True)
          acc_train_img(predictions_img, train_label)
          acc_train_text(predictions_text, train_label)


          closs = cont_loss(out_img, out_txt)
          text_loss = criterion_text(predictions_text, train_label)
          img_loss = criterion_image(predictions_img, train_label)
          batch_loss = closs*2. + text_loss*0.5 + img_loss*.15
          batch_loss.backward()
          optimizer.step()
          total_loss_train += batch_loss.item()
      
      total_acc_val = 0
      total_loss_val = 0

      with torch.no_grad():

          for val_image, val_text, val_label in val_dataloader:

              val_label = val_label.to(device)
              val_image = val_image.to(device)
              mask = val_text['attention_mask'].to(device)
              input_id = val_text['input_ids'].squeeze(1).to(device)

              out_img, out_txt, predictions_img, predictions_text = model.forward(val_image, input_id, mask, contrastive = True)

              acc_val_img(predictions_img, val_label)
              acc_val_text(predictions_text, val_label)


              closs = cont_loss(out_img, out_txt)
              text_loss = criterion_text(predictions_text, val_label)
              img_loss = criterion_image(predictions_img, val_label)
              batch_loss = closs#closs + l3
              total_loss_val += batch_loss.item()

             # acc_val(predictions, val_label)
              
      
      avg_train_loss = total_loss_train/len(train_df)
    #   train_accuracy = total_acc_train/len(train_df)

      avg_val_loss = total_loss_val/len(val_df)
    #   val_accuracy = total_acc_val/len(dev_df)

      print("Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.4f} img, {:.4f} txt".format(epoch_num+1, EPOCHS, avg_train_loss*BATCH_SIZE, acc_train_img.compute(), acc_train_text.compute()))
      print("Epoch [{}/{}], Val Loss: {:.4f}, Val Accuracy: {:.4f} img, {:.4f} txt".format(epoch_num+1, EPOCHS, avg_val_loss*BATCH_SIZE, acc_val_img.compute(), acc_val_text.compute()))

      acc_train_img.reset()
      acc_train_text.reset()
      acc_val_img.reset
      acc_val_text.reset()

      if(epoch_num%5 == 0):
        torch.save(model.state_dict(), './' + str(epoch_num+21) + '.pt' )


100%|██████████| 305/305 [01:04<00:00,  4.74it/s]


Epoch [1/25], Train Loss: 4.1680, Train Accuracy: 0.8909 img, 0.9698 txt
Epoch [1/25], Val Loss: 1.9324, Val Accuracy: 0.9059 img, 0.9914 txt


100%|██████████| 305/305 [01:10<00:00,  4.32it/s]


Epoch [2/25], Train Loss: 3.9850, Train Accuracy: 0.9163 img, 0.9955 txt
Epoch [2/25], Val Loss: 1.9174, Val Accuracy: 0.9097 img, 0.9955 txt


100%|██████████| 305/305 [01:10<00:00,  4.30it/s]


...

Epoch [4/25], Train Loss: 4.8987, Train Accuracy: 0.9450 img, 0.8994 txt
Epoch [4/25], Val Loss: 2.2408, Val Accuracy: 0.9119 img, 0.9097 txt


100%|██████████| 305/305 [01:11<00:00,  4.27it/s]


Epoch [5/25], Train Loss: 4.6718, Train Accuracy: 0.9606 img, 0.9063 txt
Epoch [5/25], Val Loss: 2.2397, Val Accuracy: 0.9115 img, 0.9097 txt


 24%|██▎       | 72/305 [00:16<00:54,  4.31it/s]


KeyboardInterrupt: 

In [ ]:
with open("27.pt", 'rb') as f:
    model.load_state_dict(torch.load(f))

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

# loss_img = nn.CrossEntropyLoss()
# loss_text = nn.CrossEntropyLoss()

# cont_loss = contrastive_loss
# criterion = nn.BCELoss()#torch.tensor([0.5, 1, 5, 5, 5, 6, 5, 1]).float().to(device))
# AVERAGING = 'micro'
# acc_train = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
# acc_val   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)

# EPOCHS = 10

# for epoch_num in range(EPOCHS):

#       total_acc_train = 0
#       total_loss_train = 0

#       for train_image, train_text, train_label in tqdm(train_dataloader):
#           optimizer.zero_grad()

#           train_label = train_label.to(device)
#           train_image = train_image.to(device)
#           mask = train_text['attention_mask'].to(device)
#           input_id = train_text['input_ids'].squeeze(1).to(device)
          
#           # logits_per_image, logits_per_text
#           #out_img, out_txt = model.forward(train_image, input_id, mask, contrastive = True)
#           predictions = model.forward(train_image, contrastive = False)

#           acc_train(predictions, train_label)

#           #ground_truth = torch.arange(len(train_image),dtype=torch.long,device=device)

#           #closs = cont_loss(out_img, out_txt)
#           l3 = criterion(predictions, train_label)
#           batch_loss = l3 #closs + l3
#           batch_loss.backward()
#           optimizer.step()
#           total_loss_train += batch_loss.item()
          
#         #   acc = (output['logits'].argmax(dim=1) == train_label).sum().item()
#         #   total_acc_train += acc
      
#       total_acc_val = 0
#       total_loss_val = 0

#       with torch.no_grad():

#           for val_image, val_text, val_label in val_dataloader:

#               val_label = val_label.to(device)
#               val_image = val_image.to(device)
#               mask = val_text['attention_mask'].to(device)
#               input_id = val_text['input_ids'].squeeze(1).to(device)

#               #out_img, out_txt = model.forward(val_image, input_id, mask, contrastive = True)
#               predictions = model.forward(val_image, contrastive = False)
#               #ground_truth = torch.arange(len(val_image),dtype=torch.long,device=device)

#               #closs = cont_loss(out_img, out_txt)
#               l3 = criterion(predictions, val_label)
#               batch_loss = l3
#               total_loss_val += batch_loss.item()

#               acc_val(predictions, val_label)
              
      
#       avg_train_loss = total_loss_train/len(train_df)
#     #   train_accuracy = total_acc_train/len(train_df)

#       avg_val_loss = total_loss_val/len(val_df)
#     #   val_accuracy = total_acc_val/len(dev_df)

#       print(f"Epoch [{epoch_num+1}/{EPOCHS}], "f"Train Loss: {avg_train_loss*BATCH_SIZE:.4f}, "f"Train Accuracy: {acc_train.compute():.4f}")
#       print(f"Epoch [{epoch_num+1}/{EPOCHS}], "f"Val Loss: {avg_val_loss*BATCH_SIZE:.4f}, "f"Val Accuracy: {acc_val.compute():.4f}")
#       print('-'*60)

#       acc_train.reset()
#       acc_val.reset()

      
#       if(epoch_num%5 == 0):
#         torch.save(model.state_dict(), './' + 'finetune' + str(epoch_num+21) + '.pt' )


In [ ]:
with open("27.pt", 'rb') as f:
    model.load_state_dict(torch.load(f))

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# criterion = nn.BCELoss(torch.tensor([1, 1, 5, 5, 5, 6, 5, 1]))
# test_loss = 0
# test_acc  = 0
# AVERAGING = AVERAGING'
# acc_train = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
# acc_val   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)

# EPOCHS = 5

# for epoch_num in range(EPOCHS):
#       total_loss_train = 0

#       for train_image, train_text, train_label in tqdm(train_dataloader):
#           optimizer.zero_grad()

#           train_label = train_label.to(device)
#           train_image = train_image.to(device)
#           mask = train_text['attention_mask'].to(device)
#           input_id = train_text['input_ids'].squeeze(1).to(device)
          
#           # logits_per_image, logits_per_text
#           out_img = model.forward(train_image, contrastive = False)
#           #print(out_img.shape, train_label.shape)
#           batch_loss = criterion(out_img, train_label)
#           batch_loss.backward()
#           optimizer.step()
#           total_loss_train += batch_loss.item()
          
#           acc_train(out_img, train_label)
#         #   acc = (output['logits'].argmax(dim=1) == train_label).sum().item()
#         #   total_acc_train += acc
      
#       total_acc_val = 0
#       total_loss_val = 0

#       with torch.no_grad():

#           for val_image, val_text, val_label in val_dataloader:

#               val_label = val_label.to(device)
#               val_image = val_image.to(device)
#               mask = val_text['attention_mask'].to(device)
#               input_id = val_text['input_ids'].squeeze(1).to(device)

#               out_img = model.forward(val_image, contrastive = False)
              

#               batch_loss = criterion(out_img, val_label)
#               total_loss_val += batch_loss.item()
#               acc_val(out_img,val_label)
              
      
#       avg_train_loss = total_loss_train/len(train_df)
#     #   train_accuracy = total_acc_train/len(train_df)

#       avg_val_loss = total_loss_val/len(val_df)
#     #   val_accuracy = total_acc_val/len(dev_df)

#       print(f"Epoch [{epoch_num+1}/{EPOCHS}], "f"Train Loss: {avg_train_loss*BATCH_SIZE:.4f}, "f"Train Accuracy: {acc_train.compute():.4f}")
#       print(f"Epoch [{epoch_num+1}/{EPOCHS}], "f"Val Loss: {avg_val_loss*BATCH_SIZE:.4f}, "f"Val Accuracy: {acc_val.compute():.4f}")
#       print('-'*60)
      
#       acc_train.reset()
#       acc_val.reset()

#       torch.save(model.state_dict(), './' + str(epoch_num)+'finetuning' + '.pt' )


## Evaluation on test set

In [ ]:
criterion = nn.BCELoss(torch.tensor([0.5, 1, 5, 5, 5, 6, 5, 1]).float().to(device))

test_loss = 0
test_acc  = 0

AVERAGING = 'weighted'
PREC = torchmetrics.classification.MultilabelPrecision(8, average = AVERAGING).to(device)#, validate_args = False)
ACC = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
REC = torchmetrics.classification.MultilabelRecall(8, average = AVERAGING).to(device)#, validate_args = False)
F1_SCORE = torchmetrics.classification.MultilabelF1Score(8, average = AVERAGING).to(device)#, validate_args = False)
F_BETA_SCORE = torchmetrics.classification.MultilabelFBetaScore(beta = 0.8, num_classes = 8, num_labels = 8, average = AVERAGING).to(device)#, validate_args = False)
KAPPA = torchmetrics.classification.MulticlassCohenKappa(8).to(device)#, validate_args = False)
AUC = torchmetrics.classification.MultilabelAUROC(8, average = AVERAGING).to(device)#, validate_args = False)

for train_image, train_text, train_label in tqdm(test_dataloader): 
    with torch.no_grad():
        train_label = train_label.to(device)
        train_image = train_image.to(device)
        mask = train_text['attention_mask'].to(device)
        input_id = train_text['input_ids'].squeeze(1).to(device)
        
        # logits_per_image, logits_per_text
        #out_img, out_txt = model.forward(train_image, input_id, mask, contrastive = True)
        predictions = model.forward(train_image, contrastive = False)



        train_label = train_label.long()
        PREC(predictions, train_label)
        ACC(predictions, train_label)
        REC(predictions, train_label)
        F1_SCORE(predictions, train_label)
        F_BETA_SCORE(predictions, train_label)
        KAPPA(predictions, train_label)
        AUC(predictions, train_label)


add_prec = PREC.compute()
add_acc = ACC.compute()
add_rec = REC.compute()
add_f1 = F1_SCORE.compute()
add_fbeta = F_BETA_SCORE.compute()
add_kappa = KAPPA.compute()
add_auc = AUC.compute()

avg_test_loss = test_loss/len(test_df)*BATCH_SIZE
avg_test_acc  = test_acc /len(test_df)

print("Acc: {:3f}\nPrec: {:3f}\nRecall: {:.3f}\nF1-score: {:.3f}\nF-Beta-score: {:.3f}\nKappa: {:.3f}\nAUC: {:.3f}".format(add_acc, add_prec,add_rec, add_f1, add_fbeta, add_kappa, add_auc))
torch.cuda.empty_cache()

  0%|          | 0/40 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\torchnew\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 40/40 [00:04<00:00,  9.10it/s]

Acc: 0.690245
Prec: 0.369007
Recall: 0.205
F1-score: 0.261
F-Beta-score: 0.278
Kappa: 0.000
AUC: 0.577
